In [20]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# データ読み込み

In [21]:
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
stations = pd.read_csv("station_list.csv")

In [22]:
#欠損値削除
train["reviews_per_month"] = train["reviews_per_month"].fillna(0)
train["last_review"] = train["last_review"].fillna("1900-01-01")
test["reviews_per_month"] = test["reviews_per_month"].fillna(0)
test["last_review"] = test["last_review"].fillna("1900-01-01")

In [23]:
!station_distance.py

distance.csv already existed


In [24]:
distance = pd.read_csv("distance.csv")
#adversarial validationのためのラベル付け
train["label"] = 1
test["label"] = 0
all_data = pd.concat([train,test],axis=0)
all_data["near_station"] = distance["near_station"]
all_data["dist_from_sta"] = distance["dist_from_sta"]
train.drop("label",axis=1,inplace=True)
test.drop("label",axis=1,inplace=True)

In [25]:
all_data.head()

,id,name,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,y,label,near_station,dist_from_sta
0,1,KiyosumiShirakawa 3min|★SkyTree★|WIFI|Max4|Tre...,242899459,Koto Ku,35.68185,139.80310,Entire home/apt,1,55,2020-04-25,2.21,173,12008.0,1,清澄白河,385.676932
1,2,Downtown Tokyo Iriya next to Ueno,308879948,Taito Ku,35.72063,139.78536,Entire home/apt,6,72,2020-03-25,2.11,9,6667.0,1,入谷,155.380783
2,3,"Japan Style,Private,Affordable,4min to Sta.",300877823,Katsushika Ku,35.74723,139.82349,Entire home/apt,1,18,2020-03-23,3.46,288,9923.0,1,堀切菖蒲園,369.710584
3,4,4 min to Shinjuku Sta. by train / 2 ppl / Wi-fi,236935461,Shibuya Ku,35.68456,139.68077,Entire home/apt,1,2,2020-04-02,1.76,87,8109.0,1,初台,626.117356
4,5,LICENSED SHINJUKU HOUSE: Heart of the action!,243408889,Shinjuku Ku,35.69840,139.70467,Entire home/apt,1,86,2020-01-30,2.00,156,100390.0,1,東新宿,265.953843


# 特徴量削除

In [26]:
id = test["id"]
drop_list = ["id","host_id","latitude","longitude"]
train.drop(drop_list,axis=1,inplace=True)
test.drop(drop_list,axis=1,inplace=True)
all_data.drop(["y"],axis=1,inplace=True)
all_data.drop(drop_list,axis=1,inplace=True)

In [27]:
all_data.columns

Index(['name', 'neighbourhood', 'room_type', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'availability_365', 'label', 'near_station', 'dist_from_sta'],
      dtype='object')

In [28]:
one_hot_list = ["room_type","neighbourhood"]
all_data = pd.get_dummies(all_data,columns=one_hot_list)

In [29]:
X_all = all_data.drop(["label"],axis=1).select_dtypes(exclude="object")
y_all = all_data["label"]
X_train_adv, X_valid_adv, y_train_adv, y_valid_adv = train_test_split(X_all, y_all, test_size=0.33, random_state=42, shuffle=True)
model = RFR()
model.fit(X_train_adv, y_train_adv)
y_pred = model.predict(X_valid_adv)
print(roc_auc_score(y_valid_adv,y_pred))

0.783053277072911


In [30]:
for column in all_data.drop("label",axis=1).select_dtypes(exclude="object").columns:
    X_all = all_data.drop(["label",column],axis=1).select_dtypes(exclude="object")
    y_all = all_data["label"]
    X_train_adv, X_valid_adv, y_train_adv, y_valid_adv = train_test_split(X_all, y_all, test_size=0.33, random_state=42, shuffle=True)
    scores = []
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    for tr_idx, va_idx in kf.split(X_all):
        tr_x, va_x = X_all.iloc[tr_idx], X_all.iloc[va_idx]
        tr_y, va_y = y_all.iloc[tr_idx], y_all.iloc[va_idx]
        model.fit(tr_x, tr_y)
        va_pred = model.predict(va_x)
        roc_auc = roc_auc_score(va_y,va_pred)
        scores.append(roc_auc)
    # クロスバリデーションの平均のスコアを出力する
    print("{0}:".format(column),np.mean(scores))

minimum_nights: 0.7448932640433191
number_of_reviews: 0.7692164737327458
reviews_per_month: 0.7771359859263282
availability_365: 0.7135844861912541
dist_from_sta: 0.7823812881375762
room_type_Entire home/apt: 0.784534686536215
room_type_Hotel room: 0.7866806298450729
room_type_Private room: 0.7864652399922545
room_type_Shared room: 0.7875391722082424
neighbourhood_Adachi Ku: 0.7864292038703422
neighbourhood_Arakawa Ku: 0.7851103178659546
neighbourhood_Bunkyo Ku: 0.7865375662805778
neighbourhood_Chiyoda Ku: 0.7844141225173311
neighbourhood_Chuo Ku: 0.7831790187767464
neighbourhood_Edogawa Ku: 0.7829323353090156
neighbourhood_Itabashi Ku: 0.7850473642457592
neighbourhood_Katsushika Ku: 0.7849866353471772
neighbourhood_Kita Ku: 0.7849060943760054
neighbourhood_Koto Ku: 0.784012311266836
neighbourhood_Meguro Ku: 0.7882114964016278
neighbourhood_Minato Ku: 0.7845866451435249
neighbourhood_Nakano Ku: 0.7848779842067656
neighbourhood_Nerima Ku: 0.7849797700542291
neighbourhood_Ota Ku: 0.78474

# 駅情報等を追加し、モデル作成

In [11]:
total_data = pd.concat([train,test],axis=0)
total_data["near_station"] = distance["near_station"]
total_data["dist_from_sta"] = distance["dist_from_sta"]
total_data = pd.get_dummies(total_data,columns=one_hot_list)
train,test = total_data[:train.shape[0]],total_data[train.shape[0]:].drop("y",axis=1)

In [12]:
train.columns

Index(['name', 'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'availability_365', 'y', 'near_station',
       'dist_from_sta', 'room_type_Entire home/apt', 'room_type_Hotel room',
       'room_type_Private room', 'room_type_Shared room',
       'neighbourhood_Adachi Ku', 'neighbourhood_Arakawa Ku',
       'neighbourhood_Bunkyo Ku', 'neighbourhood_Chiyoda Ku',
       'neighbourhood_Chuo Ku', 'neighbourhood_Edogawa Ku',
       'neighbourhood_Itabashi Ku', 'neighbourhood_Katsushika Ku',
       'neighbourhood_Kita Ku', 'neighbourhood_Koto Ku',
       'neighbourhood_Meguro Ku', 'neighbourhood_Minato Ku',
       'neighbourhood_Nakano Ku', 'neighbourhood_Nerima Ku',
       'neighbourhood_Ota Ku', 'neighbourhood_Setagaya Ku',
       'neighbourhood_Shibuya Ku', 'neighbourhood_Shinagawa Ku',
       'neighbourhood_Shinjuku Ku', 'neighbourhood_Suginami Ku',
       'neighbourhood_Sumida Ku', 'neighbourhood_Taito Ku',
       'neighbourhood_Toshima Ku'],
      dtype

In [13]:
int_columns = train.select_dtypes(exclude="object").columns
X_train,y_train = train[int_columns].drop(["y"],axis=1),train["y"]
X_test = test.select_dtypes(exclude="object")

In [14]:
X_train.columns

Index(['minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'availability_365', 'dist_from_sta', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'neighbourhood_Adachi Ku',
       'neighbourhood_Arakawa Ku', 'neighbourhood_Bunkyo Ku',
       'neighbourhood_Chiyoda Ku', 'neighbourhood_Chuo Ku',
       'neighbourhood_Edogawa Ku', 'neighbourhood_Itabashi Ku',
       'neighbourhood_Katsushika Ku', 'neighbourhood_Kita Ku',
       'neighbourhood_Koto Ku', 'neighbourhood_Meguro Ku',
       'neighbourhood_Minato Ku', 'neighbourhood_Nakano Ku',
       'neighbourhood_Nerima Ku', 'neighbourhood_Ota Ku',
       'neighbourhood_Setagaya Ku', 'neighbourhood_Shibuya Ku',
       'neighbourhood_Shinagawa Ku', 'neighbourhood_Shinjuku Ku',
       'neighbourhood_Suginami Ku', 'neighbourhood_Sumida Ku',
       'neighbourhood_Taito Ku', 'neighbourhood_Toshima Ku'],
      dtype='object')

In [15]:
#定義したX_train,y_trainを入れる
scores = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]
    model.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    rmsle = np.sqrt(mean_squared_error(np.log1p(va_y), np.log1p(va_pred)))
    scores.append(rmsle)
    print(rmsle)

# クロスバリデーションの平均のスコアを出力する
print(f'RMSLE: {np.mean(scores):.4f}')

0.8569140503427893
0.853057505769426
0.8169897242113493
0.8551292151568244
0.8287167152451475
RMSLE: 0.8422


In [17]:
model = RFR()
model.fit(X_train,y_train)
pred = model.predict(X_test)

In [18]:
submission = pd.DataFrame({"id":id,"y":pred})
submission

,id,y
0,1,14117.35
1,2,24227.83
2,3,17273.87
3,4,31651.29
4,5,15545.36
...,...,...
4991,4992,24872.12
4992,4993,8168.80
4993,4994,17846.76
4994,4995,6982.56


In [19]:
submission.to_csv("submission.csv",index=False)